## Deliverable 2. Create a Customer Travel Destinations Map.

In [1]:
# Dependencies and Setup
import pandas as pd
import requests
import gmaps

# Import API key
from config import g_key

# Configure gmaps API key
gmaps.configure(api_key=g_key)

In [2]:
# 1. Import the WeatherPy_database.csv file. 
city_data_df = pd.read_csv("../weather_database/WeatherPy_database.csv")
city_data_df.head()

,City_ID,City,Country,Date,Lat,Lng,Max Temp,Humidity,Cloudiness,Wind Speed,Current Description
0,0,Pokrovsk,UA,2022-03-30 13:06:48,48.2820,37.1758,61.92,38,100,13.82,overcast clouds
1,1,Mataura,NZ,2022-03-30 13:09:35,-46.1927,168.8643,53.55,98,100,1.19,overcast clouds
2,2,Rikitea,PF,2022-03-30 13:09:36,-23.1203,-134.9692,78.96,68,78,9.26,broken clouds
3,3,Longyearbyen,SJ,2022-03-30 13:09:36,78.2186,15.6401,12.04,62,0,5.75,clear sky
4,4,Mar Del Plata,AR,2022-03-30 13:09:36,-38.0023,-57.5575,52.88,85,40,12.66,moderate rain


In [3]:
# 2. Prompt the user to enter minimum and maximum temperature criteria 
temp_min = float(input("What is the minimum temperature you would like for your trip? "))
temp_max = float(input("What is the maximum temperature you would like for your trip? "))

What is the minimum temperature you would like for your trip? 70
What is the maximum temperature you would like for your trip? 85


In [4]:
# 3. Filter the city_data_df DataFrame using the input statements to create a new DataFrame using the loc method.
prefer_cities_possNULL_df = city_data_df.loc[(city_data_df["Max Temp"] <= temp_max) & 
                                       (city_data_df["Max Temp"] >= temp_min)]

In [5]:
# 4a. Determine if there are any empty rows.
prefer_cities_possNULL_df.isnull().sum()

City_ID                0
City                   0
Country                4
Date                   0
Lat                    0
Lng                    0
Max Temp               0
Humidity               0
Cloudiness             0
Wind Speed             0
Current Description    0
dtype: int64

In [8]:
# 4b. Drop any empty rows and create a new DataFrame that doesn’t have empty rows.
clean_df = prefer_cities_possNULL_df.dropna()
clean_df.isnull().sum()

City_ID                0
City                   0
Country                0
Date                   0
Lat                    0
Lng                    0
Max Temp               0
Humidity               0
Cloudiness             0
Wind Speed             0
Current Description    0
dtype: int64

In [9]:
# 5a. Create DataFrame called hotel_df to store hotel names along with city, country, max temp, and coordinates.
hotel_df = clean_df[["City", "Country", "Max Temp", "Current Description", "Lat", "Lng"]].copy()

# 5b. Create a new column "Hotel Name"
hotel_df["Hotel Name"] = ""
hotel_df.head()

,City,Country,Max Temp,Current Description,Lat,Lng,Hotel Name
2,Rikitea,PF,78.96,broken clouds,-23.1203,-134.9692,
9,Manaus,BR,79.29,light rain,-3.1019,-60.0250,
10,Port Alfred,ZA,71.19,overcast clouds,-33.5906,26.8910,
11,Abha,SA,73.65,few clouds,18.2164,42.5053,
18,Farafangana,MG,78.44,light rain,-22.8167,47.8333,


In [10]:
# 6a. Set parameters to search for hotels with 5000 meters.
params = {
    "radius": 5000,
    "type": "lodging",
    "key": g_key
}

# 6b. Iterate through the hotel DataFrame.
for index, row in hotel_df.iterrows():
    # 6c. Get latitude and longitude from DataFrame
    lat = row["Lat"]
    lng = row["Lng"]
    
    # Add the latitude and longitude to location key for the params dictionary.
    params["location"] = f"{lat},{lng}"
    
    # 6d. Set up the base URL for the Google Directions API to get JSON data.
    base_url = "https://maps.googleapis.com/maps/api/place/nearbysearch/json"

    # 6e. Make request and retrieve the JSON data from the search. 
    hotels = requests.get(base_url, params=params).json()
    
    # 6f. Get the first hotel from the results and store the name, if a hotel isn't found skip the city.
    try:
        hotel_df.loc[index, "Hotel Name"] = hotels["results"][0]["name"]
    except (IndexError):
        print("Hotel not found...skipping.")
        

Hotel not found...skipping.
Hotel not found...skipping.
Hotel not found...skipping.
Hotel not found...skipping.
Hotel not found...skipping.
Hotel not found...skipping.


In [14]:
# 7. Drop the rows where there is no Hotel Name.
clean_hotel_df = hotel_df.dropna()
clean_hotel_df.head()

,City,Country,Max Temp,Current Description,Lat,Lng,Hotel Name
2,Rikitea,PF,78.96,broken clouds,-23.1203,-134.9692,People ThankYou
9,Manaus,BR,79.29,light rain,-3.1019,-60.0250,Manaus Hotéis Millennium
10,Port Alfred,ZA,71.19,overcast clouds,-33.5906,26.8910,The Halyards Hotel
11,Abha,SA,73.65,few clouds,18.2164,42.5053,Abha Hotel
18,Farafangana,MG,78.44,light rain,-22.8167,47.8333,Miarina


In [17]:
# 8a. Create the output File (CSV)
output_file = "WeatherPy_Vacation.csv"
# 8b. Export the City_Data into a csv
clean_hotel_df.to_csv(output_file, index_label="City_ID")

In [27]:
# 9. Using the template add city name, the country code, the weather description and maximum temperature for the city.
info_box_template = """
<dl>
<dt>Hotel Name</dt><dd>{Hotel Name}</dd>
<dt>City</dt><dd>{City}</dd>
<dt>Country</dt><dd>{Country}</dd>
<dt>Current Weather</dt><dd>{Current Description} and {Max Temp} °F</dd>
</dl>
"""

# 10a. Get the data from each row and add it to the formatting template and store the data in a list.
hotel_info = [info_box_template.format(**row) for index, row in clean_hotel_df.iterrows()]

# 10b. Get the latitude and longitude from each row and store in a new DataFrame.
locations = clean_hotel_df[["Lat", "Lng"]]

In [28]:
# 11a. Add a marker layer for each city to the map. 
fig = gmaps.figure(center=(30.0, 31.0), zoom_level=1.5)
marker_layer = gmaps.marker_layer(locations, info_box_content=hotel_info)
fig.add_layer(marker_layer)
# 11b. Display the figure
fig

Figure(layout=FigureLayout(height='420px'))